In [1]:
import matplotlib

In [2]:
matplotlib.use('Agg')

In [3]:
%matplotlib qt

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
import SimpleITK as sitk

In [7]:
from os.path import expanduser, join

In [8]:
from scipy.spatial.distance import euclidean

In [9]:
import imgscroll

---

In [10]:
def show_imgs(imgstack, imgstack2=None):
    """Uses imgscroll to show stack of images with mouse scrolling"""
    X = sitk.GetArrayFromImage(imgstack)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    if imgstack is not None:
        Xover = sitk.GetArrayFromImage(imgstack2)
        tracker = 
    tracker = imgscroll.IndexTracker(ax, X)
    fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
    plt.show()

In [11]:
def input_level_set(featImg, seed2radius):
    """Construct input level set image from user-supplied seed points"""
    setupImg = sitk.Image(featImg.GetSize()[0], featImg.GetSize()[1], sitk.sitkUInt8)

    X2D = sitk.GetArrayFromImage(setupImg)
    for s in seed2radius.keys():
        rowIni, rowEnd = s[0] - seed2radius[s], s[0] + seed2radius[s]
        colIni, colEnd = s[1] - seed2radius[s], s[1] + seed2radius[s]
        for i in range(rowIni, rowEnd+1):
            for j in range(colIni, colEnd+1):
                if euclidean((i,j), s) <= seed2radius[s]:
                    X2D[i,j] = 1
    X = X2D.reshape(1, *X2D.shape).repeat(featImg.GetSize()[2], axis=0)
    
    img = sitk.Cast(sitk.GetImageFromArray(X), featImg.GetPixelIDValue()) * -1 + 0.5
    img.SetSpacing(featImg.GetSpacing())
    img.SetOrigin(featImg.GetOrigin())
    img.SetDirection(featImg.GetDirection())
    return img

In [58]:
class IndexTracker2(object):
    def __init__(self, ax, X, Xover):
        self.ax = ax
        self.X = X
        self.Xover = Xover
        self.slices = X.shape[0]
        self.ind = 0
        
        self.im = ax.imshow(X[self.ind,:,:], cmap=plt.cm.Greys_r)
        self.imOver = ax.imshow(Xover[self.ind,:,:], alpha=0.5)
        self.update()
        
    def onscroll(self, event):
        if event.button == 'up':
            self.ind = np.clip(self.ind+1, 0, self.slices-1)
        else:
            self.ind = np.clip(self.ind-1, 0, self.slices-1)

        self.update()
    
    def update(self):
        self.im.set_data(self.X[self.ind,:,:])
        self.imOver.set_data(self.Xover[self.ind,:,:])
        self.ax.set_title('slice %s' %self.ind)
        self.im.axes.figure.canvas.draw()
        self.imOver.axes.figure.canvas.draw()

---

# Read in DICOM images

In [12]:
dicomPath = join(expanduser('~'), 'Documents', 'SlicerDICOMDatabase', 'TCIALocal', '0', 'images', '')
reader = sitk.ImageSeriesReader()
seriesIDread = reader.GetGDCMSeriesIDs(dicomPath)[1]
dicomFilenames = reader.GetGDCMSeriesFileNames(dicomPath, seriesIDread)
reader.SetFileNames(dicomFilenames)
imgSeries = reader.Execute()

In [13]:
# choose a series of slices
imgSlices = imgSeries[:,:,41:50]

In [12]:
show_imgs(imgSlices)

Cast original slices to unsigned 8-bit integer so that segmentations can be overlaid on top:

In [41]:
imgSlicesUI8 = sitk.Cast(sitk.RescaleIntensity(imgSlices), sitk.sitkUInt8)

# Filtering

## Curvature anisotropic diffusion

In [14]:
timeStep_, conduct, numIter = (0.06, 9.0, 5)
imgRecast = sitk.Cast(imgSlices, sitk.sitkFloat32)
curvDiff = sitk.CurvatureAnisotropicDiffusionImageFilter()
curvDiff.SetTimeStep(timeStep_)
curvDiff.SetConductanceParameter(conduct)
curvDiff.SetNumberOfIterations(numIter)
imgFilter = curvDiff.Execute(imgRecast)

In [14]:
show_imgs(imgFilter)

# Edge potential

## Gradient magnitude recursive Gaussian

In [15]:
sigma_ = 2.0
imgGauss = sitk.GradientMagnitudeRecursiveGaussian(image1=imgFilter, sigma=sigma_)

In [20]:
show_imgs(imgGauss)

# Feature Image

## Sigmoid mapping

In [16]:
K1, K2 = 20.0, 8.0

In [17]:
alpha_ = (K2 - K1)/6
beta_ = (K1 + K2)/2

sigFilt = sitk.SigmoidImageFilter()
sigFilt.SetAlpha(alpha_)
sigFilt.SetBeta(beta_)
sigFilt.SetOutputMaximum(1.0)
sigFilt.SetOutputMinimum(0.0)
imgSigmoid = sigFilt.Execute(imgGauss)

In [19]:
show_imgs(imgSigmoid)

# Input Level Set

In [18]:
coords = [(125, 163)]

In [19]:
radii = [10]

In [20]:
seed2radius = {tuple(reversed(p[0])): p[1] for p in zip(coords, radii)}

In [21]:
initImg = input_level_set(imgSigmoid, seed2radius)

In [51]:
show_imgs(initImg)

In [59]:
Xsig = sitk.GetArrayFromImage(imgSigmoid)
Xinit = sitk.GetArrayFromImage(initImg)
Xmask = np.ma.masked_where(Xinit == 0.5, Xinit)

fig = plt.figure()
ax = fig.add_subplot(111)
tracker2 = IndexTracker2(ax, Xsig, Xmask)
fig.canvas.mpl_connect('scroll_event', tracker2.onscroll)

6